In [12]:
import numpy as np
import pandas as pd
import lifelines
import plotly.subplots
from sklearn import datasets
from sklearn.cluster import AgglomerativeClustering

df = pd.read_csv('data/generated/clean_train.csv')

## KAPLAN-MEIER

In [13]:
fig = plotly.subplots.make_subplots(rows=2, cols=1, shared_xaxes=True, print_grid=False, subplot_titles=("Kaplan-Meier Curve", "Patients at risk"), vertical_spacing=0.39)
kmfs = []

steps = 5 # the number of time points where number of patients at risk which should be shown

x_min = 0 # min value in x-axis, used to make sure that both plots have the same range
x_max = 0 # max value in x-axis

fig.update_layout(height=800, width=1200)

lcolors = ['#636EFA','#FF6692','#00CC96','#AB63FA']
bcolors = ['rgba(158,185,243, 0.4)','rgba(253,218,236,0.4)','rgba(179,226,205, 0.4)','rgba(222,203,228, 0.4)']
i = 0                 
for PAM50 in df.PAM50.unique():
    kmf = lifelines.KaplanMeierFitter()
    if PAM50 == 'Basal':
        TBasal = df[df.PAM50 == PAM50]["RFS"]
        EBasal = df[df.PAM50 == PAM50]["RFSE"]  
        kmf.fit(TBasal, event_observed=EBasal)

    if PAM50 == 'Her2':
        THer2 = df[df.PAM50 == PAM50]["RFS"]
        EHer2 = df[df.PAM50 == PAM50]["RFSE"]
        kmf.fit(THer2, event_observed=EHer2)

    if PAM50 == 'LumA':
        TLumA = df[df.PAM50 == PAM50][:400]["RFS"]
        ELumA = df[df.PAM50 == PAM50][:400]["RFSE"]
        kmf.fit(TLumA, event_observed=ELumA)

    if PAM50 == 'LumB':
        TLumB = df[df.PAM50 == PAM50]["RFS"]
        ELumB = df[df.PAM50 == PAM50]["RFSE"]
        kmf.fit(TLumB, event_observed=ELumB)

    kmfs.append(kmf)
    x_max = max(x_max, max(kmf.event_table.index))
    x_min = min(x_min, min(kmf.event_table.index))

    
    fig.add_trace(plotly.graph_objs.Scatter(x=kmf.survival_function_.index,
                                                y=kmf.confidence_interval_.values[:,0], line=dict(width=0), showlegend=False,
                                                name=PAM50), 
                    1, 1)
    fig.add_trace(plotly.graph_objs.Scatter(x=kmf.survival_function_.index,
                                                y=kmf.confidence_interval_.values[:,1], line=dict(width=0), showlegend=False,
                                                name=PAM50,fillcolor=bcolors[i],
    fill='tonexty',), 
                    1, 1)
    fig.append_trace(plotly.graph_objs.Scatter(x=kmf.survival_function_.index,
                                                  y=kmf.survival_function_.values.flatten(), line=dict(color=lcolors[i]),
                                                  name=PAM50), 
                     1, 1)

    i += 1
    
for s, PAM50 in enumerate(df.PAM50.unique()):
    x = []
    kmf_ = kmfs[s].event_table
    for i in range(0, int(x_max), int(x_max / (steps - 1))):
        x.append(kmf_.iloc[np.abs(kmf_.index - i).argsort()[0]].name)
    fig.append_trace(plotly.graph_objs.Scatter(x=x, 
                                               y=[PAM50 + " "] * len(x), 
                                               text=[kmfs[s].event_table[kmfs[s].event_table.index == t].at_risk.values[0] for t in x], 
                                               mode='text', 
                                               showlegend=False), 
                     2, 1)

fig.update_yaxes(title_text="Tumor", row=2, col=1)
fig.update_yaxes(title_text="Disease-specific survival probability", row=1, col=1)
fig.update_xaxes(title_text="Time (RFS)", row=2, col=1)


# just a dummy line used as a spacer/header
t = [''] * len(x)
fig.append_trace(plotly.graph_objs.Scatter(x=x, 
                                           y=[''] * (len(x)-20), 
                                           text=t,
                                           mode='text', 
                                           showlegend=False), 
                 2, 1)

    
# prettier layout
x_axis_range = [x_min - x_max * 0.05, x_max * 1.05]

fig['layout']['xaxis2']['visible'] = True
fig['layout']['xaxis2']['range'] = x_axis_range
fig['layout']['xaxis']['range'] = x_axis_range
fig['layout']['yaxis']['domain'] = [0.4, 1]
fig['layout']['yaxis2']['domain'] = [0.0, 0.3]
fig['layout']['yaxis2']['showgrid'] = False
fig['layout']['yaxis']['showgrid'] = True

fig.show()

In [14]:
from lifelines.statistics import logrank_test
results=logrank_test(THer2,TBasal, event_observed_A=EHer2, event_observed_B=EBasal)
results

<lifelines.StatisticalResult: logrank_test>
               t_0 = -1
 null_distribution = chi squared
degrees_of_freedom = 1
         test_name = logrank_test

---
 test_statistic    p  -log2(p)
           0.93 0.34      1.57

## BoxCox

In [15]:
from lifelines import CoxPHFitter
df = pd.read_csv('data/generated/clean_train.csv')

Basal = df[df.PAM50 == 'Basal']
Her2 = df[df.PAM50 == 'Her2']
LumA = df[df.PAM50 == 'LumA']
LumB = df[df.PAM50 == 'LumB']

features = [
      'Cell_Cycle',
      'HIPPO', 'MYC', 'NOTCH', 'NRF2', 'PI3K', 'TGF.Beta', 'RTK_RAS', 'TP53',
      'WNT', 'Hypoxia', 'SRC', 'ESR1', 'ERBB2', 'PROLIF','stage','grade','RFSE','RFS','age_at_diagnosis','lymph_nodes_positive','TP53.mut','PIK3CA.mut','CELLULARITY','NPI']

res = df[features]

df_basal = Basal[features]
df_her2 = Her2[features]
df_lumA = LumA[features]
df_lumB = LumB[features]
cph_Basal = CoxPHFitter()
cph_Her2 = CoxPHFitter()
cph_LumA = CoxPHFitter()
cph_LumB = CoxPHFitter()

In [16]:
cph_Basal.fit(df_basal, duration_col="RFS", event_col="RFSE")
cph_Basal.print_summary()

<lifelines.CoxPHFitter: fitted with 327 total observations, 219 right-censored observations>
             duration col = 'RFS'
                event col = 'RFSE'
      baseline estimation = breslow
   number of observations = 327
number of events observed = 108
   partial log-likelihood = -552.34
         time fit was run = 2021-06-17 16:18:56 UTC

---
                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                    
Cell_Cycle             1.33       3.78       1.47            -1.55             4.22                 0.21                67.79
HIPPO                 -2.14       0.12       2.08            -6.22             1.94                 0.00                 6.96
MYC                   -0.30       0.74       1.45            -3.15             2.55                 0.04                12.78
NOTCH                  1.08       2.94       2.50            -3.82             5.98                 0.02               394.52
NRF2                  -0.78       0.46       0.54            -1.83             0.27                 0.16                 1.31
PI3K                   1.22       3.38       1.74            -2.19             4.63                 0.11               102.13
TGF.Beta              -1.30       0.27       0.88            -3.02             0.43                 0.05                 1.54
RTK_RAS               -0.59       0.55       1.45            -3.44             2.26                 0.03                 9.57
TP53                  -0.86       0.42       0.98            -2.77             1.06                 0.06                 2.89
WNT                    2.78      16.14       2.69            -2.48             8.04                 0.08              3115.86
Hypoxia                2.96      19.31       1.87            -0.71             6.63                 0.49               757.23
SRC                    1.63       5.12       2.46            -3.19             6.46                 0.04               638.58
ESR1                   2.29       9.87       1.42            -0.48             5.06                 0.62               158.03
ERBB2                 -2.95       0.05       1.26            -5.42            -0.48                 0.00                 0.62
PROLIF                -0.52       0.59       0.63            -1.76             0.72                 0.17                 2.05
stage                 -0.16       0.85       0.13            -0.42             0.09                 0.66                 1.10
grade                 -0.15       0.86       0.40            -0.92             0.63                 0.40                 1.87
age_at_diagnosis      -0.00       1.00       0.01            -0.02             0.01                 0.98                 1.01
lymph_nodes_positive   0.06       1.06       0.04            -0.01             0.13                 0.99                 1.13
TP53.mut              -0.08       0.92       0.27            -0.60             0.44                 0.55                 1.56
PIK3CA.mut             0.54       1.71       0.28            -0.01             1.08                 0.99                 2.94
CELLULARITY           -0.05       0.95       0.15            -0.35             0.25                 0.70                 1.28
NPI                    0.57       1.77       0.21             0.17             0.97                 1.18                 2.64

                         z    p   -log2(p)
covariate                                 
Cell_Cycle            0.90 0.37       1.45
HIPPO                -1.03 0.30       1.72
MYC                  -0.21 0.84       0.26
NOTCH                 0.43 0.67       0.59
NRF2                 -1.46 0.15       2.78
PI3K                  0.70 0.48       1.05
TGF.Beta             -1.47 0.14       2.83
RTK_RAS              -0.41 0.69       0.55
TP53                 -0.88 0.38       1.39
WNT                  

In [17]:
cph_Her2.fit(df_her2, "RFS", event_col="RFSE")
cph_Her2.print_summary()

<lifelines.CoxPHFitter: fitted with 236 total observations, 141 right-censored observations>
             duration col = 'RFS'
                event col = 'RFSE'
      baseline estimation = breslow
   number of observations = 236
number of events observed = 95
   partial log-likelihood = -445.82
         time fit was run = 2021-06-17 16:18:57 UTC

---
                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                    
Cell_Cycle             0.03       1.04       1.34            -2.60             2.67                 0.07                14.41
HIPPO                  2.20       8.99       2.27            -2.25             6.64                 0.11               768.77
MYC                   -0.05       0.95       1.58            -3.14             3.04                 0.04                20.98
NOTCH                 -0.20       0.82       2.77            -5.63             5.24                 0.00               188.41
NRF2                   0.60       1.82       0.64            -0.66             1.86                 0.51                 6.44
PI3K                   0.46       1.58       2.17            -3.79             4.70                 0.02               110.28
TGF.Beta              -0.02       0.98       1.14            -2.25             2.22                 0.10                 9.18
RTK_RAS                1.80       6.04       1.40            -0.94             4.54                 0.39                93.47
TP53                  -0.62       0.54       1.02            -2.61             1.38                 0.07                 3.98
WNT                   -3.39       0.03       2.89            -9.06             2.27                 0.00                 9.68
Hypoxia                1.48       4.40       1.80            -2.05             5.01                 0.13               150.11
SRC                   -0.29       0.75       2.89            -5.96             5.39                 0.00               218.26
ESR1                   0.95       2.58       1.29            -1.59             3.48                 0.20                32.57
ERBB2                  2.65      14.21       1.65            -0.57             5.88                 0.57               357.21
PROLIF                -0.03       0.97       0.54            -1.09             1.03                 0.34                 2.80
stage                  0.02       1.02       0.13            -0.23             0.27                 0.80                 1.31
grade                 -1.01       0.37       0.33            -1.65            -0.36                 0.19                 0.70
age_at_diagnosis      -0.01       0.99       0.01            -0.03             0.01                 0.97                 1.01
lymph_nodes_positive   0.04       1.04       0.02            -0.01             0.08                 0.99                 1.09
TP53.mut               0.22       1.24       0.29            -0.35             0.79                 0.70                 2.20
PIK3CA.mut             0.02       1.02       0.23            -0.42             0.47                 0.66                 1.60
CELLULARITY           -0.13       0.88       0.19            -0.50             0.25                 0.61                 1.28
NPI                    0.66       1.94       0.18             0.30             1.02                 1.35                 2.77

                         z      p   -log2(p)
covariate                                   
Cell_Cycle            0.03   0.98       0.03
HIPPO                 0.97   0.33       1.58
MYC                  -0.03   0.98       0.03
NOTCH                -0.07   0.94       0.08
NRF2                  0.93   0.35       1.51
PI3K                  0.21   0.83       0.26
TGF.Beta             -0.02   0.99       0.02
RTK_RAS               1.29   0.20       2.33
TP53                 -0.61   0.55       0.88


In [18]:
cph_LumA.fit(df_lumA, "RFS", event_col="RFSE")
cph_LumA.print_summary()

<lifelines.CoxPHFitter: fitted with 706 total observations, 596 right-censored observations>
             duration col = 'RFS'
                event col = 'RFSE'
      baseline estimation = breslow
   number of observations = 706
number of events observed = 110
   partial log-likelihood = -615.24
         time fit was run = 2021-06-17 16:18:57 UTC

---
                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                    
Cell_Cycle            -0.68       0.51       1.08            -2.80             1.45                 0.06                 4.27
HIPPO                  1.27       3.57       1.90            -2.45             4.99                 0.09               147.66
MYC                   -0.78       0.46       1.35            -3.44             1.87                 0.03                 6.49
NOTCH                  1.42       4.15       2.25            -3.00             5.84                 0.05               344.47
NRF2                   0.12       1.12       0.53            -0.92             1.15                 0.40                 3.15
PI3K                  -3.16       0.04       1.84            -6.77             0.44                 0.00                 1.56
TGF.Beta              -0.38       0.68       0.84            -2.03             1.26                 0.13                 3.53
RTK_RAS               -0.13       0.88       1.29            -2.65             2.40                 0.07                10.98
TP53                   0.01       1.01       0.79            -1.54             1.56                 0.21                 4.77
WNT                    0.54       1.72       2.45            -4.27             5.35                 0.01               210.55
Hypoxia                2.33      10.30       2.28            -2.13             6.79                 0.12               891.73
SRC                    5.95     383.34       2.23             1.58            10.31                 4.87             30173.71
ESR1                   0.55       1.73       1.60            -2.59             3.69                 0.07                39.87
ERBB2                  4.62     101.19       1.65             1.38             7.86                 3.96              2585.88
PROLIF                 1.27       3.56       0.40             0.49             2.05                 1.63                 7.77
stage                 -0.45       0.64       0.15            -0.74            -0.16                 0.48                 0.85
grade                 -0.12       0.88       0.26            -0.63             0.38                 0.53                 1.46
age_at_diagnosis       0.01       1.01       0.01            -0.00             0.03                 1.00                 1.03
lymph_nodes_positive   0.21       1.24       0.04             0.14             0.29                 1.15                 1.33
TP53.mut               0.23       1.26       0.31            -0.38             0.85                 0.68                 2.34
PIK3CA.mut             0.30       1.35       0.21            -0.10             0.71                 0.90                 2.03
CELLULARITY           -0.07       0.93       0.17            -0.41             0.27                 0.66                 1.31
NPI                    0.11       1.11       0.15            -0.20             0.41                 0.82                 1.50

                         z      p   -log2(p)
covariate                                   
Cell_Cycle           -0.62   0.53       0.91
HIPPO                 0.67   0.50       0.99
MYC                  -0.58   0.56       0.83
NOTCH                 0.63   0.53       0.92
NRF2                  0.22   0.83       0.28
PI3K                 -1.72   0.09       3.54
TGF.Beta             -0.46   0.65       0.62
RTK_RAS              -0.10   0.92       0.12
TP53                  0.01   0.99       0.02

In [19]:
cph_LumB.fit(df_lumB, "RFS", event_col="RFSE")
cph_LumB.print_summary()

<lifelines.CoxPHFitter: fitted with 484 total observations, 336 right-censored observations>
             duration col = 'RFS'
                event col = 'RFSE'
      baseline estimation = breslow
   number of observations = 484
number of events observed = 148
   partial log-likelihood = -775.69
         time fit was run = 2021-06-17 16:18:58 UTC

---
                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                    
Cell_Cycle             0.45       1.56       0.99            -1.49             2.38                 0.23                10.85
HIPPO                 -1.62       0.20       1.68            -4.91             1.67                 0.01                 5.33
MYC                   -0.99       0.37       1.16            -3.27             1.29                 0.04                 3.63
NOTCH                  2.32      10.19       1.93            -1.46             6.11                 0.23               449.62
NRF2                  -0.61       0.54       0.51            -1.62             0.39                 0.20                 1.48
PI3K                   0.73       2.07       1.85            -2.90             4.35                 0.06                77.77
TGF.Beta               0.76       2.13       0.78            -0.77             2.29                 0.46                 9.86
RTK_RAS                0.23       1.26       1.21            -2.14             2.60                 0.12                13.42
TP53                  -1.68       0.19       0.72            -3.10            -0.26                 0.05                 0.77
WNT                    1.81       6.10       2.23            -2.56             6.17                 0.08               480.00
Hypoxia               -1.04       0.35       1.75            -4.47             2.40                 0.01                10.97
SRC                    4.52      91.48       2.05             0.51             8.53                 1.66              5050.57
ESR1                  -0.71       0.49       1.16            -2.98             1.55                 0.05                 4.71
ERBB2                  0.49       1.64       1.32            -2.08             3.07                 0.12                21.58
PROLIF                 0.66       1.93       0.39            -0.10             1.42                 0.90                 4.13
stage                  0.04       1.05       0.10            -0.15             0.24                 0.86                 1.27
grade                 -0.84       0.43       0.22            -1.27            -0.41                 0.28                 0.66
age_at_diagnosis       0.01       1.01       0.01            -0.01             0.02                 0.99                 1.02
lymph_nodes_positive   0.00       1.00       0.02            -0.04             0.04                 0.96                 1.04
TP53.mut               0.47       1.59       0.21             0.06             0.87                 1.06                 2.39
PIK3CA.mut             0.31       1.36       0.19            -0.06             0.67                 0.94                 1.96
CELLULARITY           -0.09       0.91       0.16            -0.41             0.22                 0.66                 1.25
NPI                    0.84       2.32       0.14             0.57             1.11                 1.76                 3.04

                         z      p   -log2(p)
covariate                                   
Cell_Cycle            0.45   0.65       0.62
HIPPO                -0.96   0.34       1.58
MYC                  -0.85   0.39       1.34
NOTCH                 1.20   0.23       2.12
NRF2                 -1.19   0.23       2.11
PI3K                  0.39   0.69       0.53
TGF.Beta              0.97   0.33       1.59
RTK_RAS               0.19   0.85       0.24
TP53                 -2.32   0.02       5.61